# Dynamic Parking Pricing System

This notebook implements a real-time dynamic pricing system for urban parking lots, featuring three pricing models, robust data handling, and interactive Bokeh visualization.

## Table of Contents
- [1. Imports & Setup](#setup)
- [2. Data Processing](#data)
- [3. Pricing Models](#models)
- [4. Simulation & Results](#simulation)
- [5. Visualization (Bokeh)](#viz)


In [10]:
%pip install pandas numpy bokeh notebook

Note: you may need to restart the kernel to use updated packages.


In [11]:
# 1. Imports & Setup
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

try:
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.models import ColumnDataSource, HoverTool, Div, DataTable, TableColumn
    from bokeh.layouts import column as bokeh_column, row as bokeh_row
    from bokeh.palettes import Category20
    output_notebook()
except ImportError:
    print("Bokeh not installed. Install with: pip install bokeh")

Loading BokehJS ...

## 2. Data Processing

Handles loading, mapping, and preprocessing your parking dataset.

In [12]:
class DataProcessor:
    def __init__(self, csv_path="dataset.csv"):
        self.csv_path = csv_path
        self.base_price = 10.0
    
    def load_dataset(self):
        df = pd.read_csv(self.csv_path)
        print(f"Loaded {len(df)} records. Columns: {list(df.columns)}")
        return df
    
    def map_columns(self, df):
        mapping = {
            'ID': 'space_id',
            'Capacity': 'capacity',
            'Latitude': 'latitude',
            'Longitude': 'longitude',
            'Occupancy': 'occupancy',
            'VehicleType': 'vehicle_type',
            'TrafficConditionNearby': 'traffic_level',
            'QueueLength': 'queue_length',
            'IsSpecialDay': 'is_special_day',
            'LastUpdatedDate': 'date',
            'LastUpdatedTime': 'time'
        }
        df = df.rename(columns=mapping)
        # Parse datetime robustly
        dt_str = df['date'] + ' ' + df['time']
        try:
            df['timestamp'] = pd.to_datetime(dt_str, format='%d-%m-%Y %H:%M:%S')
        except:
            df['timestamp'] = pd.to_datetime(dt_str, dayfirst=True, errors='coerce')
        df = df.drop(['date', 'time'], axis=1)
        return df
    
    def preprocess(self, df):
        df = self.map_columns(df)
        df = df.dropna(subset=['timestamp'])
        df = df.sort_values('timestamp')
        if df['space_id'].min() == 1:
            df['space_id'] -= 1
        df['occupancy_rate'] = df['occupancy'] / df['capacity']
        df['hour'] = df['timestamp'].dt.hour
        df['day_of_week'] = df['timestamp'].dt.dayofweek
        vehicle_map = {'Car': 'car', 'Bike': 'bike', 'Truck': 'truck', 'Motorcycle': 'bike', 'Bus': 'truck'}
        df['vehicle_type'] = df['vehicle_type'].map(lambda x: vehicle_map.get(str(x).capitalize(), 'car'))
        vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
        df['vehicle_weight'] = df['vehicle_type'].map(vehicle_weights)
        if df['traffic_level'].dtype == 'object':
            traffic_map = {'Low': 1, 'Medium': 5, 'High': 10, 'Light': 2, 'Moderate': 6, 'Heavy': 9, 'Severe': 10}
            df['traffic_level'] = df['traffic_level'].map(lambda x: traffic_map.get(str(x).capitalize(), 5))
        df['traffic_level'] = np.clip(df['traffic_level'], 1, 10)
        if df['is_special_day'].dtype == 'object':
            df['is_special_day'] = df['is_special_day'].map(lambda x: str(x).lower() in ['true', 'yes', '1'])
        return df.reset_index(drop=True)

## 3. Pricing Models

Three models: Baseline, Demand-Based, and Competitive.

In [13]:
# Baseline Model
class BaselineLinearModel:
    def __init__(self, base_price=10.0, alpha=0.5):
        self.base_price = base_price
        self.alpha = alpha
        self.current_prices = {}
    def initialize(self, space_ids):
        for sid in space_ids:
            self.current_prices[sid] = self.base_price
    def update_price(self, sid, occupancy_rate):
        price = self.current_prices.get(sid, self.base_price)
        price += self.alpha * occupancy_rate
        price = max(self.base_price * 0.5, min(price, self.base_price * 2.0))
        self.current_prices[sid] = price
        return price

# Demand-Based Model
class DemandBasedModel:
    def __init__(self, base_price=10.0):
        self.base_price = base_price
        self.current_prices = {}
        self.alpha = 0.6; self.beta = 0.3; self.gamma = 0.2
        self.delta = 0.4; self.epsilon = 0.1; self.lambda_param = 0.8
    def initialize(self, space_ids):
        for sid in space_ids:
            self.current_prices[sid] = self.base_price
    def update_price(self, sid, occupancy_rate, queue_length, traffic_level, is_special_day, vehicle_weight):
        demand = (self.alpha * occupancy_rate +
                  self.beta * min(queue_length / 20.0, 1.0) -
                  self.gamma * (traffic_level - 1) / 9.0 +
                  self.delta * float(is_special_day) +
                  self.epsilon * vehicle_weight)
        norm_demand = np.tanh(demand)
        price = self.base_price * (1 + self.lambda_param * norm_demand)
        price = max(self.base_price * 0.5, min(price, self.base_price * 2.0))
        self.current_prices[sid] = price
        return price

# Competitive Model
class CompetitivePricingModel:
    def __init__(self, base_price=10.0):
        self.base_price = base_price
        self.current_prices = {}
        self.locations = {}
        self.alpha = 0.6; self.beta = 0.3; self.gamma = 0.2
        self.delta = 0.4; self.epsilon = 0.1; self.lambda_param = 0.8
        self.competition_radius = 0.005
        self.competition_weight = 0.3
        self.demand_weight = 0.7
    def initialize(self, space_ids, locations):
        for sid in space_ids:
            self.current_prices[sid] = self.base_price
        self.locations = locations
    def _distance(self, lat1, lon1, lat2, lon2):
        return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)
    def _competitors(self, sid):
        loc = self.locations[sid]
        return [other_sid for other_sid, other_loc in self.locations.items()
                if other_sid != sid and self._distance(loc['latitude'], loc['longitude'], other_loc['latitude'], other_loc['longitude']) <= self.competition_radius]
    def update_price(self, sid, occupancy_rate, queue_length, traffic_level, is_special_day, vehicle_weight, occupancy, capacity):
        demand = (self.alpha * occupancy_rate +
                  self.beta * min(queue_length / 20.0, 1.0) -
                  self.gamma * (traffic_level - 1) / 9.0 +
                  self.delta * float(is_special_day) +
                  self.epsilon * vehicle_weight)
        norm_demand = np.tanh(demand)
        demand_price = self.base_price * (1 + self.lambda_param * norm_demand)
        competitors = self._competitors(sid)
        if competitors:
            avg_competitor_price = np.mean([self.current_prices.get(cid, self.base_price) for cid in competitors])
            price_ratio = avg_competitor_price / self.current_prices[sid]
            comp_adj = (price_ratio - 1) * 0.2 if occupancy < capacity else -0.1
            comp_price = self.current_prices[sid] * (1 + comp_adj)
            final_price = self.demand_weight * demand_price + self.competition_weight * comp_price
        else:
            final_price = demand_price
        final_price = max(self.base_price * 0.5, min(final_price, self.base_price * 2.0))
        self.current_prices[sid] = final_price
        return final_price

## 4. Simulation & Results

Runs all models on your dataset and collects results.

In [14]:
# Load and preprocess data
processor = DataProcessor("dataset.csv")
df = processor.load_dataset()
df = processor.preprocess(df)

# Initialize models
space_ids = sorted(df['space_id'].unique())
locations = {sid: {'latitude': df[df['space_id']==sid]['latitude'].iloc[0],
                  'longitude': df[df['space_id']==sid]['longitude'].iloc[0]} for sid in space_ids}

baseline = BaselineLinearModel()
baseline.initialize(space_ids)
demand = DemandBasedModel()
demand.initialize(space_ids)
competitive = CompetitivePricingModel()
competitive.initialize(space_ids, locations)

# Run simulation
results = []
for idx, row in df.iterrows():
    sid = int(row['space_id'])
    occ_rate = float(row['occupancy_rate'])
    qlen = int(row['queue_length'])
    tlev = int(row['traffic_level'])
    is_special = bool(row['is_special_day'])
    vweight = float(row['vehicle_weight'])
    occupancy = int(row['occupancy'])
    capacity = int(row['capacity'])
    timestamp = row['timestamp']
    b_price = baseline.update_price(sid, occ_rate)
    d_price = demand.update_price(sid, occ_rate, qlen, tlev, is_special, vweight)
    c_price = competitive.update_price(sid, occ_rate, qlen, tlev, is_special, vweight, occupancy, capacity)
    results.append({
        'timestamp': timestamp,
        'space_id': sid,
        'occupancy_rate': occ_rate,
        'queue_length': qlen,
        'traffic_level': tlev,
        'is_special_day': is_special,
        'baseline_price': b_price,
        'demand_price': d_price,
        'competitive_price': c_price
    })
    if idx % 1000 == 0:
        print(f"Processed {idx} rows...")

results_df = pd.DataFrame(results)

Loaded 18368 records. Columns: ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']
Processed 0 rows...
Processed 1000 rows...
Processed 2000 rows...
Processed 3000 rows...
Processed 4000 rows...
Processed 5000 rows...
Processed 6000 rows...
Processed 7000 rows...
Processed 8000 rows...
Processed 9000 rows...
Processed 10000 rows...
Processed 11000 rows...
Processed 12000 rows...
Processed 13000 rows...
Processed 14000 rows...
Processed 15000 rows...
Processed 16000 rows...
Processed 17000 rows...
Processed 18000 rows...


## 5. Visualization (Bokeh)

Interactive charts for model comparison and insight.

In [15]:
def plot_dashboard(results_df):
    # Time series plot
    avg_by_time = results_df.groupby('timestamp').agg({
        'baseline_price': 'mean',
        'demand_price': 'mean',
        'competitive_price': 'mean'
    }).reset_index()
    p1 = figure(title="Model Comparison Over Time", x_axis_type='datetime', width=900, height=350)
    p1.line(avg_by_time['timestamp'], avg_by_time['baseline_price'], color='blue', legend_label='Baseline', line_width=2)
    p1.line(avg_by_time['timestamp'], avg_by_time['demand_price'], color='red', legend_label='Demand-Based', line_width=2)
    p1.line(avg_by_time['timestamp'], avg_by_time['competitive_price'], color='green', legend_label='Competitive', line_width=2)
    p1.legend.location = 'top_left'
    p1.xaxis.axis_label = 'Time'; p1.yaxis.axis_label = 'Average Price ($)'

    # Space-wise bar plot
    avg_by_space = results_df.groupby('space_id').agg({
        'baseline_price': 'mean',
        'demand_price': 'mean',
        'competitive_price': 'mean'
    }).reset_index()
    p2 = figure(title="Average Price by Space", x_axis_label='Space ID', y_axis_label='Avg Price ($)', width=700, height=350)
    p2.vbar(x=avg_by_space['space_id']-0.2, top=avg_by_space['baseline_price'], width=0.2, color='blue', legend_label='Baseline')
    p2.vbar(x=avg_by_space['space_id'], top=avg_by_space['demand_price'], width=0.2, color='red', legend_label='Demand-Based')
    p2.vbar(x=avg_by_space['space_id']+0.2, top=avg_by_space['competitive_price'], width=0.2, color='green', legend_label='Competitive')
    p2.legend.location = 'top_left'

    # Occupancy vs price scatter
    sample = results_df.sample(n=min(1000, len(results_df)))
    p3 = figure(title="Occupancy vs Price (Demand Model)", x_axis_label='Occupancy Rate (%)', y_axis_label='Price ($)', width=700, height=350)
    p3.scatter(sample['occupancy_rate']*100, sample['demand_price'], color='red', alpha=0.5)

    show(bokeh_column(p1, bokeh_row(p2, p3)))
plot_dashboard(results_df)